# **Sentiment Analysis on Tweets**
Sentiment analysis, also known as opinion mining, is the process of identifying and categorizing emotions expressed in text data—typically as positive, negative, or neutral. It helps organizations and individuals understand the sentiment behind user-generated content, such as product reviews, social media posts, or customer feedback.

In the context of social media, sentiment analysis is particularly valuable due to the vast amount of real-time user opinions shared daily. Twitter, with its concise and public messages, provides an ideal dataset for analyzing public sentiment around topics, events, brands, or products.

By leveraging natural language processing (NLP) techniques and machine learning models, sentiment analysis can extract insights from tweets to support business decisions, brand monitoring, political analysis, and crisis management.

This project focuses on building a sentiment classifier using a dataset of tweets. The model aims to classify each tweet as positive or negative, helping reveal how people feel about certain topics at scale.

<br>

**Dataset:** [Kaggle Sentiment140](https://www.kaggle.com/datasets/kazanova/sentiment140/data)

---

## **Data Preprocessing**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import emoji

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Juls\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Juls\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Juls\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Juls\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Juls\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Prepare the dataset.

In [2]:
dataset_path = '../data/twt.csv'
column_names = ['sentiment', 'id', 'date', 'flag', 'user', 'text']
df = pd.read_csv(dataset_path, encoding='latin', delimiter=',', names=column_names)
df.head()

,sentiment,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Drop unimportant columns and missing values.

In [3]:
df = df.drop(['id', 'date', 'flag', 'user'], axis=1)
df = df.dropna()
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


### Preprocess the data:
- convert the emojis into text
- lowercase everything
- remove urls, mentions and hashtags
- remove punctuations and special characters
- remove stopwords
- split into tokens

In [4]:
def clean_twts(twt):
    twt = twt.lower()  # Lowercase
    twt = re.sub(r"http\S+|www\S+|https\S+", '', twt)  # Remove URLs
    twt = re.sub(r"@\w+", '', twt)  # Remove mentions
    twt = re.sub(r"#", '', twt)  # Remove hashtag symbol, keep the word
    twt = emoji.demojize(twt)  # Convert emojis to text
    twt = re.sub(r"[^a-zA-Z\s]", '', twt)  # Remove punctuations and special characters
    twt = re.sub(r"\s+", ' ', twt).strip()  # Remove extra spaces
    tokens = twt.split()  # Tokenize
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    
    return ' '.join(tokens)

cleaned_twts = df['text'].apply(clean_twts)
df['cleaned_text'] = cleaned_twts

In [5]:
df.head()

,sentiment,text,cleaned_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",thats bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset cant update facebook texting might cry r...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving im mad cant see


Tokenize the data.

In [6]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(cleaned_twts)
sequences = tokenizer.texts_to_sequences(cleaned_twts)
padded_sequences = pad_sequences(sequences, maxlen=30, padding='post', truncating='post')

print("Tokenized and padded sequences:")
print(padded_sequences)

Tokenized and padded sequences:
[[  43 1089 3300 ...    0    0    0]
 [ 664   12  432 ...    0    0    0]
 [   1  212  248 ...    0    0    0]
 ...
 [ 117 7962 7937 ...    0    0    0]
 [  54  201  173 ...    0    0    0]
 [  54    1    0 ...    0    0    0]]


Add into the dataframe.

In [7]:
df['padded_text'] = list(padded_sequences)
df['sentiment'] = df['sentiment'].map({4: 1, 0: 0})
df.head()

,sentiment,text,cleaned_text,padded_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",thats bummer shoulda got david carr third day,"[43, 1089, 3300, 14, 753, 9571, 1723, 4, 0, 0,..."
1,0,is upset that he can't update his Facebook by ...,upset cant update facebook texting might cry r...,"[664, 12, 432, 436, 1881, 191, 416, 2155, 74, ..."
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds,"[1, 212, 248, 1210, 1497, 781, 361, 7, 1, 0, 0..."
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,"[320, 703, 369, 2745, 6, 1046, 0, 0, 0, 0, 0, ..."
4,0,"@nationwideclass no, it's not behaving at all....",behaving im mad cant see,"[1, 2, 480, 12, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


Split the dataset.

In [8]:
x = padded_sequences
y = df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
df

,sentiment,text,cleaned_text,padded_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",thats bummer shoulda got david carr third day,"[43, 1089, 3300, 14, 753, 9571, 1723, 4, 0, 0,..."
1,0,is upset that he can't update his Facebook by ...,upset cant update facebook texting might cry r...,"[664, 12, 432, 436, 1881, 191, 416, 2155, 74, ..."
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds,"[1, 212, 248, 1210, 1497, 781, 361, 7, 1, 0, 0..."
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,"[320, 703, 369, 2745, 6, 1046, 0, 0, 0, 0, 0, ..."
4,0,"@nationwideclass no, it's not behaving at all....",behaving im mad cant see,"[1, 2, 480, 12, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...
1599995,1,Just woke up. Having no school is the best fee...,woke school best feeling ever,"[239, 74, 93, 104, 132, 0, 0, 0, 0, 0, 0, 0, 0..."
1599996,1,TheWDB.com - Very cool to hear old Walt interv...,thewdbcom cool hear old walt interviews,"[1, 114, 178, 139, 1, 3806, 0, 0, 0, 0, 0, 0, ..."
1599997,1,Are you ready for your MoJo Makeover? Ask me f...,ready mojo makeover ask details,"[117, 7962, 7937, 500, 1872, 0, 0, 0, 0, 0, 0,..."
1599998,1,Happy 38th Birthday to my boo of alll time!!! ...,happy th birthday boo alll time tupac amaru sh...,"[54, 201, 173, 391, 4809, 15, 1, 1, 1, 0, 0, 0..."
